# Process data for LED display

Scales and adjusts LED intensity values and converts to 'uint8'

In [1]:
import os
import csv
import datetime
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [2]:
image_dir = 'images'
data_dir = 'data'

label = 'seq_data'

In [3]:
#dt_string = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
filename = f'{label}_ids.csv'
with open(os.path.join(data_dir, filename), 'r') as f:
    csv_reader = csv.reader(f, delimiter=',')
    lines = [line for line in csv_reader]

print("sequence data on file:")
sequences = []
for  line in lines:
    print(f"{line[0]}: {line[1]}")
    sequences.append(line[1])

sequence data on file:
2022-01-01 22:16: 7
2022-01-01 22:16: 7
2022-01-01 22:17: 1
2022-01-01 22:25: 2
2022-01-01 22:31: 3
2022-01-01 22:37: 4
2022-01-01 22:43: 5


In [4]:
selected_seqs = [1, 2, 3, 4, 5]

In [5]:
rgb_data = []
for c in range(3):
    filename = f'{label}_rgb_{c}.csv'
    with open(os.path.join(data_dir, filename), 'r') as f:
        x = np.loadtxt(f, delimiter=',')
    rgb_data.append(x)
assert(all(len(data) == len(sequences) for data in rgb_data))

In [6]:
# Multiply by 64

adj_params = {
    'mul': [64, 64, 32],
    'power': [1, 1, 1],
    'div': [1, 1, 1],
    'v_min': [0, 0, 0],
    'v_max': [64, 64, 32]
}

rgb_data_adj = []
for c in range(3):
    mul = adj_params['mul'][c]
    div = adj_params['div'][c]
    power = adj_params['power'][c]
    v_min = adj_params['v_min'][c]
    v_max = adj_params['v_max'][c]
    x_data = []
    for seq in selected_seqs:
        x = np.clip(rgb_data[c][seq] * mul ** power / div, 
                    v_min, v_max)
        x = x.astype('uint8')
        x_data.append(x)
    rgb_data_adj.append(x_data)

len(rgb_data_adj)

3

In [7]:
# Print output for insertion into fire_data.h file

for c in range(3):
    items = []
    print(f"\n// Data for {['R', 'G', 'B'][c]}\n")
    for seq, _ in enumerate(selected_seqs):
        data_to_strs = [f"{x:d}" for x in rgb_data_adj[c][seq].tolist()]
        items.append(', '.join(data_to_strs))
    print(',\n\n'.join(items))
    


// Data for R

2, 33, 24, 21, 20, 17, 17, 36, 58, 45, 45, 45, 26, 22, 24, 34, 35, 17, 28, 45, 37, 15, 17, 4, 24, 4, 21, 34, 28, 4, 19, 12, 2, 12, 4, 21, 27, 4, 12, 15, 10, 4, 2, 21, 54, 43, 54, 41, 60, 60, 35, 33, 60, 17, 51, 52, 60, 41, 39, 51, 54, 46, 60, 43, 43, 52, 60, 60, 60, 41, 46, 60, 21, 60, 60, 54, 60, 44, 32, 46, 59, 39, 60, 58, 54, 39,

15, 33, 25, 21, 27, 47, 30, 37, 51, 43, 45, 29, 24, 22, 25, 19, 23, 29, 25, 38, 22, 13, 15, 19, 19, 15, 21, 24, 34, 17, 18, 12, 7, 13, 16, 16, 21, 9, 9, 14, 9, 16, 9, 22, 55, 45, 53, 41, 63, 63, 34, 41, 61, 15, 50, 49, 57, 43, 37, 52, 56, 48, 59, 62, 47, 15, 58, 63, 57, 42, 39, 52, 53, 61, 63, 62, 63, 42, 29, 39, 54, 54, 59, 44, 24, 15,

15, 33, 24, 19, 18, 15, 27, 46, 53, 15, 57, 39, 24, 22, 23, 15, 32, 34, 27, 19, 14, 14, 15, 15, 19, 15, 14, 40, 21, 17, 18, 12, 7, 13, 14, 15, 18, 13, 13, 14, 9, 12, 11, 22, 57, 49, 56, 43, 63, 63, 39, 42, 63, 15, 49, 52, 58, 42, 37, 51, 56, 46, 54, 61, 43, 36, 60, 62, 59, 43, 34, 59, 56, 61, 62, 59, 44, 15